<a href="https://colab.research.google.com/github/FadelYang/craft-it-object-detection-model/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing required Libraries
# !pip install tensorflow-addons
import os
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.style as style
from google.colab import files
from sklrean.utils import class_weights

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Preparing Dataset

## Mount to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/'Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


## Load Dataset



In [4]:
# !rm -r dataset
# !rm data
# !wget https://app.roboflow.com/ds/gzGSPSpsan?key=CrzyeSBgrQ
# !mv gzGSPSpsan?key=CrzyeSBgrQ data
# !mkdir dataset
# !unzip data -d dataset

In [5]:
## Load label dataset.
dataset_dir = os.path.join(os.getcwd(), 'dataset')
train_label_df = pd.read_csv(os.path.join(dataset_dir, 'train', '_classes.csv'))
validation_label_df = pd.read_csv(os.path.join(dataset_dir, 'valid', '_classes.csv'))
train_images_dir = os.path.join(dataset_dir, 'train')
validation_images_dir = os.path.join(dataset_dir, 'valid')

print(train_images_dir)

/content/drive/MyDrive/Colab Notebooks/dataset/train


In [6]:
print(train_label_df.head())
print(validation_label_df.head())

                                           filename   aluminiumfoil   bottle  \
0  1196_jpg.rf.68689b8efc68113f5ffa321d8c02fe0b.jpg               0        0   
1   757_jpg.rf.67b5d7ce20e152ea0f2b8783ffe65301.jpg               0        0   
2   344_jpg.rf.6725ba8501f47157d433edc3e1420a85.jpg               0        1   
3    65_jpg.rf.65d079f8e8e3840c836b3902febc65c8.jpg               0        0   
4   414_jpg.rf.665baa402c9f84c0416955ebe0ad3925.jpg               0        0   

    bottlecap   can   carton   cup   paper   plastic   straw  
0           0     0        0     0       0         0       1  
1           0     0        0     1       0         0       0  
2           0     1        0     0       0         0       0  
3           1     0        0     0       0         0       0  
4           0     1        0     0       0         0       0  
                                          filename   aluminiumfoil   bottle  \
0  121_jpg.rf.030d2a2ac6f40bab2e9bea71c50b7716.jpg            

In [7]:
# !rm -r train

In [8]:
train_label_df.columns = train_label_df.columns.str.strip()
validation_label_df.columns = validation_label_df.columns.str.strip()

In [9]:
# # Check the data.
# import os

# train_image_filenames = train_label_df['filename'].tolist()
# validation_image_filenames = validation_label_df['filename'].tolist()

# train_image_paths = [os.path.join(train_images_dir, fname) for fname in train_image_filenames]
# validation_image_paths = [os.path.join(validation_images_dir, fname) for fname in validation_image_filenames]

# # Check if all files exist
# missing_train_images = [img for img in train_image_paths if not os.path.exists(img)]
# missing_validation_images = [img for img in validation_image_paths if not os.path.exists(img)]

# print(f"Missing train images: {missing_train_images}")
# print(f"Missing validation images: {missing_validation_images}")

In [10]:
columns = [
    "aluminiumfoil",
    "bottle",
    "bottlecap",
    "can",
    "carton",
    "cup",
    "paper",
    "plastic",
    "straw"
]

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)


if os.path.exists(train_images_dir):
  train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_label_df,
      directory=train_images_dir,
      # target_size=(224, 224),
      x_col='filename',
      y_col=columns,
      batch_size=32,
      seed=42,
      class_mode='raw',
      shuffle=True,
)

if os.path.exists(validation_images_dir):
  validation_generator = validation_datagen.flow_from_dataframe(
      dataframe=validation_label_df,
      directory=validation_images_dir,
      # target_size=(224, 224),
      x_col='filename',
      y_col=columns,
      batch_size=32,
      class_mode='raw',
      seed=42,
      shuffle=False,
)

Found 2331 validated image filenames.
Found 227 validated image filenames.


# Create Model and Training

## Build Model



In [11]:
feature_extractor = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=tf.keras.Input(shape=(640, 480, 3))
)

In [12]:
# Freeze layers in ResNet50
feature_extractor.trainable = False

print(feature_extractor)
print(train_generator)

In [13]:
x = feature_extractor.output.shape
print(x)

(None, 20, 15, 1280)


In [18]:
inputs = tf.keras.layers.Input(shape=(640, 480, 3))
x = feature_extractor(inputs)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(9, activation='sigmoid')(x)

# Create the model using the functional API
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [21]:
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc'), tfa.metrics.F1Score(num_classes=1, average='micro', name='f1_score')]
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
)

Epoch 1/30
73/73 [==============================] - 26s 285ms/step - loss: 0.3294 - auc: 0.7515 - f1_score: 0.3524 - val_loss: 0.3500 - val_auc: 0.6065 - val_f1_score: 0.2026
Epoch 2/30
73/73 [==============================] - 19s 260ms/step - loss: 0.3162 - auc: 0.7814 - f1_score: 0.3849 - val_loss: 0.3522 - val_auc: 0.6183 - val_f1_score: 0.2203
Epoch 3/30
73/73 [==============================] - 18s 252ms/step - loss: 0.3023 - auc: 0.8057 - f1_score: 0.4231 - val_loss: 0.3562 - val_auc: 0.6190 - val_f1_score: 0.2599
Epoch 4/30
73/73 [==============================] - 19s 265ms/step - loss: 0.2912 - auc: 0.8242 - f1_score: 0.4536 - val_loss: 0.3562 - val_auc: 0.6363 - val_f1_score: 0.2555
Epoch 5/30
73/73 [==============================] - 19s 254ms/step - loss: 0.2777 - auc: 0.8433 - f1_score: 0.4889 - val_loss: 0.3579 - val_auc: 0.6398 - val_f1_score: 0.2203
Epoch 6/30
73/73 [==============================] - 19s 256ms/step - loss: 0.2631 - auc: 0.8652 - f1_score: 0.5194 - val_loss

In [ ]:
def learning_curves(history):
    """Plot the learning curves of loss and macro f1 score
    for the training and validation datasets.

    Args:
        history: history callback of fitting a tensorflow keras model
    """

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    auc = history.history['auc']
    val_auc = history.history['val_auc']

    f1s = history.history['f1_score']
    val_f1s = history.history['val_f1_score']

    epochs = len(loss)

    style.use("bmh")
    plt.figure(figsize=(10, 15))

    plt.subplot(3, 1, 1)
    plt.plot(range(1, epochs+1), loss, label='Training Loss')
    plt.plot(range(1, epochs+1), val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')

    plt.subplot(3, 1, 2)
    plt.plot(range(1, epochs+1), auc, label='Training AUC')
    plt.plot(range(1, epochs+1), val_auc, label='Validation AUC')
    plt.legend(loc='upper right')
    plt.ylabel('AUC')
    plt.title('Training and Validation AUC')


    plt.subplot(3, 1, 3)
    plt.plot(range(1, epochs+1), f1s, label='Training F1-score')
    plt.plot(range(1, epochs+1), val_f1s, label='Validation F1-score')
    plt.legend(loc='lower right')
    plt.ylabel('F1-score')
    plt.title('Training and Validation F1-score')
    plt.xlabel('epoch')

    plt.tight_layout()
    plt.show()

    return loss, val_loss, auc, val_auc, f1s, val_f1s

In [ ]:
loss, val_loss, auc, val_auc, f1s, val_f1s = learning_curves(history)